In [1]:
import numpy as np
import os

In [2]:
def detect_namespace(namespace:str):
    content = namespace[namespace.find("(")+1:-1].split(", ")
    arg_dict = {}
    for arg in content:
        arg_type, arg_value = arg.split("=")
        arg_dict[arg_type] = arg_value
    return arg_dict

def get_acc(line:str, delimiter=":", precision=4):
    index = line.find(delimiter) + len(delimiter)
    acc = line[index:index + 3 + precision]
    return float(acc)

def parse_file(f):
    for line in f:
        if line.find("Namespace") != -1:
            arg_dict = detect_namespace(line)
        if line.find("No mask noise average acc") != -1:
            NM = get_acc(line)
        if line.find("Finetune noise average acc") != -1:
            FT = get_acc(line)
    return arg_dict, NM, FT

In [3]:
class PResultItem():
    def __init__(self, main_key:float):
        self.main_key = main_key
        self.NM_list = []
        self.FT_list = []
    
    def add_result(self, NM:float, FT:float):
        assert len(self.NM_list) == len(self.FT_list)
        self.NM_list.append(NM)
        self.FT_list.append(FT)

class PResult():
    def __init__(self):
        self.dict = {}
    
    def add_result(self, arg_dict:dict, NM:float, FT:float, main_key_str="mask_p"):
        # print(arg_dict)
        main_key = float(arg_dict[main_key_str])
        if not(main_key in self.dict.keys()):
            self.dict[main_key] = PResultItem(main_key)
        self.dict[main_key].add_result(NM, FT)

In [5]:
dir_path = "./results/pDiff200"
file_blob = os.listdir(dir_path)
main_key_str = "mask_p"
# main_key_str = "train_epoch"
results = PResult()
for fn in file_blob:
    cfn = os.path.join(dir_path, fn)
    with open(cfn) as f:
        f = f.read().split("\n")
    arg_dict, NM, FT = parse_file(f)
    results.add_result(arg_dict, NM, FT, main_key_str)

In [6]:
for j in np.sort(list(results.dict.keys())):
    i = results.dict[j]
    NM = np.mean(i.NM_list)
    FT = np.mean(i.FT_list)
    gain= FT - NM
    print(f"{main_key_str} = {i.main_key:e}, NM: {NM:.4f}, FT: {FT:.4f}, Gain: {gain:.4f}")

mask_p = 1.000000e-02, NM: 0.9294, FT: 0.9289, Gain: -0.0004
mask_p = 3.000000e-02, NM: 0.9338, FT: 0.9348, Gain: 0.0010
mask_p = 4.000000e-02, NM: 0.9322, FT: 0.9326, Gain: 0.0004
mask_p = 5.000000e-02, NM: 0.9342, FT: 0.9353, Gain: 0.0011
mask_p = 6.000000e-02, NM: 0.9226, FT: 0.9231, Gain: 0.0004
mask_p = 7.000000e-02, NM: 0.9292, FT: 0.9313, Gain: 0.0021
mask_p = 1.000000e-01, NM: 0.9330, FT: 0.9332, Gain: 0.0003


In [7]:
trace_head = "./results/traces"
a = os.listdir(trace_head)
print(a)

['saved_A_1622586201.1394565.pt', 'saved_A_1622588934.2849271.pt', 'saved_A_1622590543.5536091.pt', 'saved_A_1622590543.553758.pt', 'saved_A_1622590543.55377.pt', 'saved_A_1622590543.5538652.pt', 'saved_A_1622590546.2475152.pt', 'saved_A_1622590873.7518249.pt', 'saved_A_1622592009.025987.pt', 'saved_A_1622592814.259459.pt', 'saved_B_1622586201.1394565.pt', 'saved_B_1622588934.2849271.pt', 'saved_B_1622590543.5536091.pt', 'saved_B_1622590543.553758.pt', 'saved_B_1622590543.55377.pt', 'saved_B_1622590543.5538652.pt', 'saved_B_1622590546.2475152.pt', 'saved_B_1622590873.7518249.pt', 'saved_B_1622592009.025987.pt', 'saved_B_1622592814.259459.pt']


In [19]:
import torch
state_dict = torch.load(trace_head + "/saved_A_1622586201.1394565.pt", map_location="cpu")

In [21]:
state_dict.keys()

odict_keys(['fc1.weightS', 'fc1.op.weight', 'fc1.op.bias', 'fc2.weightS', 'fc2.op.weight', 'fc2.op.bias', 'fc3.weightS', 'fc3.op.weight', 'fc3.op.bias', 'fc4.weightS', 'fc4.op.weight', 'fc4.op.bias'])